In [1]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np
import vectorbtpro as vbt

In [2]:
prediction_window = 75

df = pd.read_csv('../results/RID0028_LSTM_pw75_lb250_bt2000_mem5000lstm_model_results.csv', index_col=0, parse_dates=True)
keep_cols = ['BTCUSDT_Open', 'BTCUSDT_High', 'BTCUSDT_Low',
       'BTCUSDT_Close','long_minus_short', 'long_slope', 'short_slope',]
df=df[keep_cols]
df.tail(3)

,BTCUSDT_Open,BTCUSDT_High,BTCUSDT_Low,BTCUSDT_Close,long_minus_short,long_slope,short_slope
close_time,,,,,,,
2023-06-16 16:50:00+00:00,25830.0,25843.1,25818.0,25835.9,1.298384,-0.001052,0.001010
2023-06-16 16:55:00+00:00,25841.8,25854.0,25841.0,25849.5,0.675644,0.000583,-0.000673
2023-06-16 16:56:00+00:00,25849.5,25989.0,25848.6,25941.4,1.213235,-0.000997,0.000436


In [3]:
# Create rolling averages for each of the following columns: long_minus_short	long_slope	short_slope
rolling_df = df.copy()
window=5

rolling_df['long_minus_short'] = df['long_minus_short'].rolling(window=window).mean()
rolling_df['long_slope'] = df['long_slope'].rolling(window=window).mean()
rolling_df['short_slope'] = df['short_slope'].rolling(window=window).mean()
rolling_df.dropna(inplace=True)
rolling_df.tail(3)

,BTCUSDT_Open,BTCUSDT_High,BTCUSDT_Low,BTCUSDT_Close,long_minus_short,long_slope,short_slope
close_time,,,,,,,
2023-06-16 16:50:00+00:00,25830.0,25843.1,25818.0,25835.9,1.055051,-0.000949,0.000787
2023-06-16 16:55:00+00:00,25841.8,25854.0,25841.0,25849.5,1.073514,-0.000696,0.000545
2023-06-16 16:56:00+00:00,25849.5,25989.0,25848.6,25941.4,1.140575,-0.000691,0.000496


In [4]:
def lms_slope_type_3(long_minus_short, long_slope, short_slope, lms_threshold, long_slope_thresh, short_slope_thresh):
    entries       = pd.Series(np.where((long_minus_short < lms_threshold) & (long_slope > long_slope_thresh), True, False))
    short_entries = pd.Series(np.where((long_minus_short < lms_threshold) & (short_slope < short_slope_thresh), True, False))
    
    return entries, short_entries

# Create an indicator factory
lms_slope_type_3_indicator = vbt.IndicatorFactory(
    class_name  ='LongMinusShortSlopeType3', # name of the class
    short_name  ='lmsSlope3', # name of the indicator
    input_names =['long_minus_short', 'long_slope', 'short_slope'], # names of input arguments
    param_names =['lms_threshold', 'long_slope_thresh', 'short_slope_thresh'], # names of parameters
    output_names=['entries', 'short_entries'], # names of output values
).with_apply_func(
    lms_slope_type_3, # function to apply
    takes_1d=True, # whether the function takes 1-dim. arrays as input
    lms_threshold=0.5, # default value for parameter 'lms_threshold'
    long_slope_thresh=0.0, # default value for parameter 'long_slope_thresh'
    short_slope_thresh=0.0, # default value for parameter 'short_slope_thresh'
)

In [6]:
num_increments         = 30

lms_min                 = rolling_df.long_minus_short.min() # To narrow the range Try 0.6
lms_max                 = rolling_df.long_minus_short.max() # and Try 1.1
long_slope_min          = rolling_df.long_slope.min()
long_slope_max          = rolling_df.long_slope.max()
short_slope_min         = rolling_df.short_slope.min()
short_slope_max         = rolling_df.short_slope.max()

def lms_slope_type_4(long_minus_short, long_slope, short_slope, lms_threshold, long_slope_thresh, short_slope_thresh):
    entries       = pd.Series(np.where((long_minus_short < lms_threshold) & (long_slope > long_slope_thresh), True, False))
    exits         = pd.Series(np.where((long_minus_short > lms_threshold*1.2), True, False))
    
    short_entries = pd.Series(np.where((long_minus_short < lms_threshold) & (short_slope < short_slope_thresh), True, False))
    short_exits   = pd.Series(np.where((long_minus_short > lms_threshold*1.2), True, False))
    
    return entries, exits, short_entries, short_exits

# Create an indicator factory
lms_slope_type_4_indicator = vbt.IndicatorFactory(
    class_name  ='LongMinusShortSlopeType4', # name of the class
    short_name  ='lmsSlope4', # name of the indicator
    input_names =['long_minus_short', 'long_slope', 'short_slope'], # names of input arguments
    param_names =['lms_threshold','long_slope_thresh', 'short_slope_thresh'], # names of parameters
    output_names=['entries', 'exits', 'short_entries', 'short_exits'], # names of output values
).with_apply_func(
    lms_slope_type_4, # function to apply
    takes_1d=True, # whether the function takes 1-dim. arrays as input
    lms_threshold=0.7, # default value for parameter 'lms_threshold'
    # lms_lower_threshold=0.4, # default value for parameter 'lms_lower_threshold'
    long_slope_thresh=0.0, # default value for parameter 'long_slope_thresh'
    short_slope_thresh=0.0, # default value for parameter 'short_slope_thresh'
)
lms_4_strategy = lms_slope_type_4_indicator.run(
    long_minus_short    =rolling_df['long_minus_short'],
    long_slope          =rolling_df['long_slope'],
    short_slope         =rolling_df['short_slope'],
    lms_threshold       =np.linspace(lms_min, lms_max, num_increments), 
    long_slope_thresh   =np.linspace(long_slope_min, long_slope_max, num_increments),
    short_slope_thresh  =np.linspace(short_slope_min, short_slope_max, num_increments),
    param_product=True, # True: all combinations of parameters, False: only one combination for each parameter
    execute_kwargs=dict(
        engine="threadpool",
        chunk_len="auto",
        show_progress=True,
    )
)

multiple_pf = vbt.Portfolio.from_signals(
    close               =rolling_df['BTCUSDT_Close'],
    high                =rolling_df['BTCUSDT_High'],
    low                 =rolling_df['BTCUSDT_Low'],
    open                =rolling_df['BTCUSDT_Open'],
    entries             =lms_4_strategy.entries,
    exits               =lms_4_strategy.exits,
    short_entries       =lms_4_strategy.short_entries,
    short_exits         =lms_4_strategy.short_exits,
    # td_stop             =prediction_window,
    # time_delta_format   ='Rows',
    accumulate          =False,
    
)

print(multiple_pf.stats()) # Prints the average of all of the simulations


  0%|          | 0/2250 [00:00<?, ?it/s]

Start                         2022-01-09 00:12:00+00:00
End                           2023-06-16 16:56:00+00:00
Period                                           115477
Start Value                                       100.0
Min Value                                     75.622867
Max Value                                    140.344648
End Value                                    104.968177
Total Return [%]                               4.968177
Benchmark Return [%]                         -37.400097
Total Time Exposure [%]                        50.28231
Max Gross Exposure [%]                       127.857064
Max Drawdown [%]                              37.796763
Max Drawdown Duration                      74614.150196
Total Orders                                1012.426333
Total Fees Paid                                     0.0
Total Trades                                 650.333963
Win Rate [%]                                  53.052606
Best Trade [%]                                12